# <font color='black'> Регрессионный анализ панельных данных и каузальность, 2026 </font>
## <font color='black'> Практическое занятие 2. Модели с фиксированными эффектами (FE-models) на временные периоды. Фиксированные эффекты и на время, и на пространство. Модель с разными наклонами  </font>
Панельные данные имеют как пространственное, так и временное измерение. К примеру, наблюдения для одной страны представлены за несколько лет. При такой структуре данных наивно было бы полагать, что условие о независимости наблюдений соблюдается. Наблюдения "внутри" одной пространственной единицы, разумеется, связаны. В этом случае мы не можем применять классическую линейную регрессию. В качестве альтернативы выступают широко применяемые модели с фиксированными эффектами.

Кратко о данных, с которыми мы сегодня будем работать.

*Источник: Массив данных основан на идее исследования Ehrlich I., Lui F. (1999) Bureaucratic Corruption and Endogenous Economic Growth. The Journal of Political Economy, 107 (6), pp. 270 – 293.*

* country -  Страна
* countrygroup - Страны в датасете разделены на три группы. Первая группа (countrygroup = 1) включает Австралию, Канаду, Японию, Новую Зеландию и США. Вторая группа (countrygroup = 2) включает страны Восточной Европы (Болгария, Хорватия, Чешская Республика, Эстония, Венгрия, Латвия, Польша, Словения, Словакия). В третью группу в основном входят страны Западной Европы
* year - Год. Данные охватывают временной промежуток с 2006 по 2015 год
* pol_stab - Политическая стабильность и отсутствие насилия/терроризма. Отражает склонность к политической нестабильности и/или политически мотивированному насилию, включая терроризм. Значения показателя варьируются от −2.5 до 2.5, причем более высокие значения означают более высокий уровень политической стабильности. Источником данных являются Индикаторы качества государственного управления (WGI). В рамках анализа будем использовать этот показатель в качестве зависимой переменной
* con_cor - Показатель контроля коррупции. Отражает восприятие степени использования государственной власти для личной выгоды, включая как мелкие, так и крупные формы коррупции. Показатель изменяется от −2.5 до 2.5, причем более высокие зна-чения соответствуют более низкому уровню коррупции (т.е. более высокому контролю). Источником данных является WGI. В рамках анализа будем использовать эту переменную в качестве ключевого предиктора
* herfgov_DPI - Индекс концентрации Херфиндаля. В контексте данного исследования он используется для измерения уровня конкуренции между политическими партиями. Шкала варьируется от 0 до 1, где 1 означает отсутствие конкуренции. В рамках анализа будем использовать как контрольную переменную
* govt_consump_WDI - Государственные расходы на конечное потребление (в процентах от ВВП). В рамках анализа будем использовать как контрольную переменную


Подгрузим необходимые библиотеки и откроем массив "RAPDC_lab1.dta".

In [30]:
import pandas as pd
import statsmodels.formula.api as statf
import numpy as np
import numpy.linalg as la
!pip install linearmodels
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from scipy import stats

In [31]:
dta = pd.read_stata("RAPDC_lab1.dta")
dta = dta.dropna()
dta.head()

,year,country,pol_stab,con_cor,herfgov_DPI,govt_consump_WDI,countrygroup
0,2006,Australia,0.935188,1.960568,0.75987,17.288634,1.0
1,2007,Australia,0.928790,2.010918,0.75987,17.175808,1.0
2,2008,Australia,0.955645,2.042482,1.00000,17.096951,1.0
3,2009,Australia,0.855689,2.051661,1.00000,17.496081,1.0
4,2010,Australia,0.888860,2.031455,1.00000,17.959095,1.0


На прошлом занятии мы уже оценивали модель с фиксированными эффектами на страны, используя внутригрупповое преобразование. Давайте вспомним результаты оценивания данной модели:

In [32]:
dta_index = dta.set_index(['country', 'year'])

In [33]:
m_country_FE = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects', data = dta_index).fit(cov_type='clustered', cluster_entity=True, group_debias=True)
print(m_country_FE)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.0833
Estimator:                   PanelOLS   R-squared (Between):              0.1598
No. Observations:                 339   R-squared (Within):               0.0833
Date:                Mon, Jan 26 2026   R-squared (Overall):              0.1589
Time:                        22:17:39   Log-likelihood                    221.05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.1488
Entities:                          34   P-value                           0.0000
Avg Obs:                       9.9706   Distribution:                   F(3,302)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             1.5347
                            

В следующей модели учтем фиксированные эффекты на временные периоды. Для начала используем LSDV-запись (то есть, модель с набором дамми-переменных)

* Проинтерпретируйте хотя бы один коэффициент при временном периоде
* Нужны ли в данном случае FE на временные периоды?
* Что теперь отражают оценки коэффициентов при предикторах: как эта интерпретация отличается от интерпретации результатов модели с фиксированными эффектами на страны? На основании какой дисперсии получены оценки этих коэффициентов?

In [34]:
m_year_FE = statf.ols(formula = 'pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + C(year)', data = dta).fit(cov_type = "HC3")
print(m_year_FE.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.446
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     26.63
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           1.39e-41
Time:                        22:17:39   Log-Likelihood:                -78.353
No. Observations:                 339   AIC:                             182.7
Df Residuals:                     326   BIC:                             232.4
Df Model:                          12                                         
Covariance Type:                  HC3                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.8492      0.098  

Переоценим модель с кластеризованными стандартными ошибками. Кластеризацию оставим на страны, так как здесь правдоподобно, что ошибки связаны между собой во времени

In [35]:
m_year_clustered_se = m_year_FE.get_robustcov_results(cov_type='cluster', groups=dta['country'])
print(m_year_clustered_se.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.446
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     9.658
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           1.15e-07
Time:                        22:17:39   Log-Likelihood:                -78.353
No. Observations:                 339   AIC:                             182.7
Df Residuals:                     326   BIC:                             232.4
Df Model:                          12                                         
Covariance Type:              cluster                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.8492      0.209  

Переоценим модель, используя внутригрупповое преобразование:

In [36]:
m_time_within = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + TimeEffects', dta_index).fit(cov_type="clustered", cluster_entity=True)
print(m_time_within)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.4427
Estimator:                   PanelOLS   R-squared (Between):              0.1148
No. Observations:                 339   R-squared (Within):               0.0222
Date:                Mon, Jan 26 2026   R-squared (Overall):              0.1139
Time:                        22:17:39   Log-likelihood                   -78.353
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      86.329
Entities:                          34   P-value                           0.0000
Avg Obs:                       9.9706   Distribution:                   F(3,326)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             15.273
                            

Оценим twoways модель: учтем и FE на год, и FE на страны. В чем заключаются ограничения этой модели?

Оценим стандартные ошибки с кластеризацией на временные периоды и страны (cluster_entity=True, cluster_time = True)

In [37]:
m_twoways = PanelOLS.from_formula('pol_stab ~ con_cor + govt_consump_WDI + herfgov_DPI + EntityEffects + TimeEffects', dta_index).fit(cov_type="clustered", cluster_entity=True, cluster_time = True)
print(m_twoways)

                          PanelOLS Estimation Summary                           
Dep. Variable:               pol_stab   R-squared:                        0.0611
Estimator:                   PanelOLS   R-squared (Between):              0.3129
No. Observations:                 339   R-squared (Within):               0.0817
Date:                Mon, Jan 26 2026   R-squared (Overall):              0.3087
Time:                        22:17:39   Log-likelihood                    226.77
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.3513
Entities:                          34   P-value                           0.0003
Avg Obs:                       9.9706   Distribution:                   F(3,293)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             0.7350
                            

Проверим, одинаковая ли взаимосвязь коррупции и политической стабильности в разных группах стран. Для этого оценим модель с разными наклонами (varying-slope model). Проинтерпретируйте полученные результаты.  

Примечание: до этого мы рассматривали страну в качестве пространственной единицы. Однако в таком случае есть опасность получить чрезмерно фрагментарные результаты, что будет сопровождаться безумным количеством параметров и сложностями в интерпретации. Вместо этого оценим модель с разными наклонами на группы стран.

In [38]:
m_varying_slopes = statf.ols('pol_stab ~ con_cor*C(countrygroup) + govt_consump_WDI + herfgov_DPI', data = dta).fit()
m_varying_slopes_clustered = m_varying_slopes.get_robustcov_results(cov_type='cluster', groups=dta['country'])
print(m_varying_slopes_clustered.summary())

                            OLS Regression Results                            
Dep. Variable:               pol_stab   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     17.20
Date:                Mon, 26 Jan 2026   Prob (F-statistic):           2.36e-09
Time:                        22:17:39   Log-Likelihood:                -48.994
No. Observations:                 339   AIC:                             114.0
Df Residuals:                     331   BIC:                             144.6
Df Model:                           7                                         
Covariance Type:              cluster                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   